In [1]:
import sys
sys.path.append('..')
from src.models.pipeline import pipeline
import glob
import json
import os
import pandas as pd
import ast
from datetime import datetime
from shutil import copyfile
from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
from bayes_opt.util import load_logs
from src.models.train_model import functionsBuilder
from src.models.audio import downloadAudioFromYoutube
import warnings
warnings.filterwarnings("ignore")

In [2]:
groundbase_dir = '../data/raw/groundbase'
transcripts_dir = os.path.join(groundbase_dir,'transcripts')
topic_dataset_path = os.path.join(groundbase_dir,'dataset.csv')
transcript_filespath = glob.glob(groundbase_dir + '/transcripts/*.json')

'''Read the transcript'''
transcripts_jsons = {}
for fl in transcript_filespath:
    with open(fl,encoding="utf8") as f:
        transcript =ast.literal_eval(f.read()) #json.load(f)
        vid = fl.split('\\')[-1].split('.')[0]
        #print(vid)
        transcripts_jsons[vid] = transcript
#print(transcripts_jsons)

'''Read the videos metadata to perform on them the segmentation'''
df_videos = pd.read_csv(topic_dataset_path)

''' Transfer topic shifts time to seconds units instead HH:MM:SS'''

def topic_shifts_seconds(topic_shifts):
    tp_shift_sec=[]
    for tp in topic_shifts:
        intervals = tp.split(':')
        seconds = int(intervals[2])
        minutes = int(intervals[1]) * 60
        hours = int(intervals[0]) * 60 *60
        tp_shift_sec.append(seconds + minutes + hours)
    return tp_shift_sec


for video_id in transcripts_jsons.keys():    
    df_videos.at[df_videos['video id'] == video_id,'topic shifts(ends)'] =\
    topic_shifts_seconds(\
                         df_videos[df_videos['video id'] == \
                                   video_id]['topic shifts(ends)'])

Divide the dataset into 3 groups ( sd,audio,image) as correspoding to each model.
we want to  prepare to use the k-folds. Each group is a Test set of the type of the model group (sd,audio,image), the rest is the training dataset. 

each model is comprise of 3 different pipelines:
    None filter
    median 2,2
    median 3,3


In [9]:
models_workflow = {
    'sliding_window':[
        'sliding_window-tfidf-cosine-None-spectral_clustering',
        'sliding_window-tfidf-cosine-median_(2,2)-spectral_clustering',
        "sliding_window-tfidf-cosine-median_(3,3)-spectral_clustering"
    ]#,
    #'audio':[
    #    'audio-tfidf-cosine-None-spectral_clustering',
    #    'audio-tfidf-cosine-median_(2,2)-spectral_clustering',
    #    'audio-tfidf-cosine-median_(3,3)-spectral_clustering'
    #]#,
    #'image' : [
        
    #]
}
models = list(models_workflow.keys())
videos_ids = df_videos.drop_duplicates(subset=['video id'])['video id'].values.tolist()
#print(videos_ids)



In [4]:
'''Downloading wav files'''
"""if 'audio' in models:
    audio_path = '../data/raw/audio'
    for vid in videos_ids:
        if os.path.isfile(os.path.join(audio_path,vid+'.wav')) is False:
            url = (str(df_videos.loc[df_videos['video id'] == vid,'youtube link'].values.tolist()[0]))
            downloadAudioFromYoutube(url)"""

"if 'audio' in models:\n    audio_path = '../data/raw/audio'\n    for vid in videos_ids:\n        if os.path.isfile(os.path.join(audio_path,vid+'.wav')) is False:\n            url = (str(df_videos.loc[df_videos['video id'] == vid,'youtube link'].values.tolist()[0]))\n            downloadAudioFromYoutube(url)"

In [4]:
'''Finding Test dataset of each model'''

## probably k folds is not relevant here

'''step = int(len(videos_ids)/len(models))
test_datasets = [videos_ids[i:i + step] for i in range(0,len(videos_ids),step)]
# adding the modulu to the last group
if len(test_datasets[-1]) < len(models):
    test_datasets[-2] = test_datasets[-2] + test_datasets[-1]
    test_datasets.pop()
    
#print(test_datasets)
#print(len(test_datasets))
'''Building the Training and Test Set'''    
dataset = [
    {
        "train":[test_datasets[i] for i in range(len(test_datasets)) if i!=mdl_index],
        "test":test_datasets[mdl_index],
        'model':models[mdl_index]
    }
    for mdl_index in range(len(models))]'''

In [5]:
'''Creating the experiment uses'''
'''date_now = datetime.now().strftime("%d-%m-%Y %H-%M-%S")
exp_path = os.path.join('../models/bayesian_opt/experiment',date_now)
os.mkdir(exp_path)
file = 'parameters_bounds.json'
parameter_path = exp_path +'/' + file
copyfile('../models/bayesian_opt/poc/' + file,parameter_path)'''

'../models/bayesian_opt/experiment\\13-05-2020 12-40-08/parameters_bounds.json'

# *Configure now the parameters bounds as you wish*

In [5]:
'''Read the parameters bounds'''
with open(os.path.join('../models/bayesian_opt/experiment','parameters_bounds.json'),'r') as f:
    param_bounds = ast.literal_eval(f.read())
df_results = pd.read_csv('../data/processed/bayesian_opt/results.csv') #pd.DataFrame(columns=['video','max_target','workflow','params','param_bounds'])

In [6]:
df_results.head()
#param_bounds

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,video,max_target,workflow,params,param_bounds
0,0,0.0,0.0,MkiUBJcgdUY,0.222222,sliding_window-tfidf-cosine-None-spectral_clus...,"{'n_clusters': 14.502132028215444, 'sim_thresh...","{""window_size"": (20,150),""step_size"":(10,60),'..."
1,1,1.0,1.0,MkiUBJcgdUY,0.222222,sliding_window-tfidf-cosine-None-spectral_clus...,"{'n_clusters': 12.880535344902679, 'sim_thresh...","{""window_size"": (20,150),""step_size"":(10,60),'..."
2,2,2.0,2.0,MkiUBJcgdUY,0.444444,sliding_window-tfidf-cosine-None-spectral_clus...,"{'n_clusters': 15.53458736722668, 'sim_thresh'...","{""window_size"": (20,150),""step_size"":(10,60),'..."
3,3,3.0,3.0,MkiUBJcgdUY,0.277778,sliding_window-tfidf-cosine-None-spectral_clus...,"{'n_clusters': 12.0, 'sim_thresh': 0.300000010...","{""window_size"": (20,150),""step_size"":(10,60),'..."
4,4,4.0,4.0,MkiUBJcgdUY,0.222222,sliding_window-tfidf-cosine-None-spectral_clus...,"{'n_clusters': 13.425649132739782, 'sim_thresh...","{""window_size"": (20,150),""step_size"":(10,60),'..."


# Training the model

In [11]:
videos_ids = ['7kLHJ-F33GI','7snJ1mx1EMQ','RIawrYLVdIw']

Train the models. Each video is running couple of times for bayesian opitmum.

In [12]:
for vid in videos_ids:
    print("running on video %s" %(vid))
    groundbase = df_videos.loc[df_videos['video id'] == vid,'topic shifts(ends)'].values.tolist()[:-1]
    tr = transcripts_jsons[vid]
    for model in models:
        #samples = [d["train"] for d in dataset if d[model] == model][0]
        for workflow_label in models_workflow[model]:
            print('Running workflow %s' %(workflow_label))
            #logger = JSONLogger(path=os.path.join(exp_path,('%s.json' %(workflow_label))))
            #optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)
            
            function_to_optimized = functionsBuilder(groundbase=groundbase,transcripts=tr).build_f_to_optimize(workflow_label)
            
            optimizer = BayesianOptimization(
                f=function_to_optimized,
                pbounds=param_bounds[workflow_label],
                verbose=2,
                random_state=1
                )
            
            optimizer.maximize(
                init_points = 2,
                n_iter = 10
            )
            
            print(optimizer.max)
            for res in optimizer.res:
                df_results = df_results.append({'video':vid,
                                                'max_target':res['target'],
                                                'workflow':workflow_label,
                                                'params':res['params'],
                                                'param_bounds':param_bounds[workflow_label]
                                               },ignore_index=True)
        

running on video 7kLHJ-F33GI
Running workflow sliding_window-tfidf-cosine-None-spectral_clustering
|   iter    |  target   | n_clus... | sim_th... | step_size | window... |
-------------------------------------------------------------------------
|  1        |  0.7273   |  14.5     |  0.5161   |  10.01    |  59.3     |
|  2        |  0.6364   |  12.88    |  0.3277   |  19.31    |  64.92    |
|  3        |  0.6364   |  18.0     |  0.6      |  10.0     |  150.0    |
label 1 has done already cannot evaluate unsequentional segmentation, try other parameters
|  4        |  0.0      |  12.0     |  0.3      |  10.0     |  20.0     |
|  5        |  0.6364   |  16.53    |  0.3012   |  10.17    |  59.85    |
|  6        |  0.3636   |  12.0     |  0.6      |  60.0     |  136.0    |
label 2 has done already cannot evaluate unsequentional segmentation, try other parameters
|  7        |  0.0      |  12.0     |  0.6      |  60.0     |  20.0     |
|  8        |  0.5455   |  12.0     |  0.6      |  10

label 3 has done already cannot evaluate unsequentional segmentation, try other parameters
|  8        |  0.0      |  15.45    |  0.334    |  59.95    |  114.7    |
|  9        |  0.4286   |  12.0     |  0.3      |  10.0     |  107.0    |
label 0 has done already cannot evaluate unsequentional segmentation, try other parameters
|  10       |  0.0      |  18.0     |  0.6      |  29.28    |  98.85    |
label 1 has done already cannot evaluate unsequentional segmentation, try other parameters
|  11       |  0.0      |  12.0     |  0.3      |  53.54    |  150.0    |
n_samples=17 should be >= n_clusters=18
|  12       |  0.0      |  18.0     |  0.6      |  60.0     |  72.89    |
{'target': 0.8571428571428571, 'params': {'n_clusters': 16.432431412456257, 'sim_thresh': 0.3000063890009115, 'step_size': 59.99514192872911, 'window_size': 21.1099397077962}}
Running workflow sliding_window-tfidf-cosine-median_(3,3)-spectral_clustering
|   iter    |  target   | n_clus... | sim_th... | step_size | w

Check the model on the Training set

In [13]:
df_results.to_csv('../data/processed/bayesian_opt/results.csv')

Examine bias and variance

In [9]:
''' Calculate bias'''
Ex = df_results.groupby(["workflow",'video'])['max_target'].mean()
Fx = 1
Ex =Ex.apply(lambda x: Fx-x)
Ex

workflow                                                      video      
sliding_window-tfidf-cosine-None-spectral_clustering          2mC1uqwEmWQ    0.573333
                                                              8BeXwhljq2g    0.733333
                                                              B-Xe7_mf2CY    0.788889
                                                              BiLJmhQMbVg    1.000000
                                                              MkiUBJcgdUY    0.722222
                                                              Q-HugPvA7GQ    0.828571
                                                              SCvtxjpVQms    0.953846
                                                              dkAr9ThdSUU    0.600000
                                                              ksh6OMRFroI    0.516667
                                                              tORLeHHtazM    1.000000
                                                              x5zL

In [12]:
'''Calculating variance'''
df_results.groupby(["workflow",'video'])['max_target'].var()

workflow                                                      video      
sliding_window-tfidf-cosine-None-spectral_clustering          2mC1uqwEmWQ    0.063556
                                                              8BeXwhljq2g    0.133333
                                                              B-Xe7_mf2CY    0.016049
                                                              BiLJmhQMbVg    0.000000
                                                              MkiUBJcgdUY    0.009259
                                                              Q-HugPvA7GQ    0.027041
                                                              SCvtxjpVQms    0.010651
                                                              dkAr9ThdSUU    0.050865
                                                              ksh6OMRFroI    0.195833
                                                              tORLeHHtazM    0.000000
                                                              x5zL